In [1]:
from SmartApi.smartConnect import SmartConnect
import pyotp
import pandas as pd
import requests
import pandas as pd
from datetime import datetime, timedelta
from datetime import datetime, timedelta
import pandas_ta as ta


api_key = 'C46AImmE'
clientId = 'M507690'
pwd = '4663'
smartApi = SmartConnect(api_key)
token = "4ACNNSIZL6EV6UQ2ACSI23H5VQ"
totp=pyotp.TOTP(token).now()
correlation_id = "abc123"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

#fetch User Profile
userProfile= smartApi.getProfile(refreshToken)
User= pd.DataFrame(userProfile)
User = User['data']['name']
User
print(f"Welcome {User} to the World of SMART API BY ANGEL.....!!")
print(f"Login Sucessfull....!!")


[I 251221 20:02:14 smartConnect:124] in pool


Welcome MANA  BHATTACHARYYA to the World of SMART API BY ANGEL.....!!
Login Sucessfull....!!


In [2]:
# ---- NIFTY 50 Token (Angel One Fixed Token) ----
NIFTY_TOKEN = "26000"      # NIFTY 50 Index
EXCHANGE = "NSE"

# ---- Fetch LTP ----
ltp_data = smartApi.ltpData(
    exchange=EXCHANGE,
    tradingsymbol="NIFTY",
    symboltoken=NIFTY_TOKEN
)

print("📈 NIFTY 50 LTP:", ltp_data['data']['ltp'])

ATM = round(ltp_data['data']['ltp'] / 50) * 50
print("📈 NIFTY 50 SPOT Price:", ATM)

ltp =  ltp_data['data']['ltp']
prev_close = ltp_data['data']['close']

change = ltp - prev_close
pct_change = (change / prev_close) * 100

print(f"NIFTY Change: {change:.2f}")
print(f"NIFTY % Change: {pct_change:.2f}%")


📈 NIFTY 50 LTP: 25966.4
📈 NIFTY 50 SPOT Price: 25950
NIFTY Change: 150.85
NIFTY % Change: 0.58%


In [3]:
import pandas as pd
import pandas_ta as ta

def add_indicators(df):
    df["VWAP"] = (df["Close"] * df["Volume"]).cumsum() / df["Volume"].cumsum()
    df["EMA14"] = ta.ema(df["Close"], length=20)
    df["RSI"] = ta.rsi(df["Close"], length=14)

    macd = ta.macd(df["Close"])
    df["MACD"] = macd["MACD_12_26_9"]
    df["MACD_SIGNAL"] = macd["MACDs_12_26_9"]

    return df

In [4]:
# ---------- FETCH 5-MIN CANDLES ---------- #
to_date = datetime.now()
from_date = to_date - timedelta(days=5)

params = {
    "exchange": "NSE",
    "symboltoken": "99926000",
    "interval": "THREE_MINUTE",
    "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
    "todate": to_date.strftime("%Y-%m-%d %H:%M")
}

data = smartApi.getCandleData(params)
NIFTY = pd.DataFrame(
    data["data"],
    columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]
)
NIFTY["Datetime"] = pd.to_datetime(NIFTY["Datetime"])
NIFTY['Datetime'] = pd.to_datetime(NIFTY['Datetime']).dt.tz_localize(None)
NIFTY['NIFTY_Change'] = NIFTY['Close'].diff()

NIFTY = add_indicators(NIFTY)
#NIFTY = NIFTY.tail(3)
NIFTY.tail(2)

,Datetime,Open,High,Low,Close,Volume,NIFTY_Change,VWAP,EMA14,RSI,MACD,MACD_SIGNAL
373,2025-12-19 15:24:00,25970.5,25973.65,25961.95,25965.65,0,-4.55,NaN,25968.793796,46.974384,-0.092907,0.369601
374,2025-12-19 15:27:00,25965.5,25972.20,25955.60,25961.40,0,-4.25,NaN,25968.089625,43.031123,-0.648065,0.166068


In [5]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
NIFTY.reset_index()

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.05,
    subplot_titles=("NIFTY Price + VWAP + Buy/Sell Signals", "RSI")
)

fig.add_trace(
    go.Scatter(
        x=NIFTY.index,                # ✅ index only → curve unchanged
        y=NIFTY['Close'],
        name='Close',
        customdata=NIFTY['Datetime'],           # 👈 attach datetime here
        hovertemplate=
            "<b>Date:</b> %{customdata}<br>" +
            "<b>Close:</b> %{y}<br>" +
            "<extra></extra>",
        line=dict(color='white', width=2)
    ),
    row=1, col=1
)


fig.update_layout(
    height=900,
    template='plotly_dark',
    title="NIFTY Option Buying Strategy • VWAP + RSI + VIX Filter",
    xaxis_rangeslider_visible=False,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()
